In [1]:
import tensorflow as tf
import numpy as np
import gym

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)
print(gym.__version__)

2.8.0
0.21.0


c:\Users\37103\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment ALE/Tetris-v5
  logger.warn("Overriding environment {}".format(id))
c:\Users\37103\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment ALE/Tetris-ram-v5
  logger.warn("Overriding environment {}".format(id))
c:\Users\37103\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment ALE/Adventure-v5
  logger.warn("Overriding environment {}".format(id))
c:\Users\37103\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment ALE/Adventure-ram-v5
  logger.warn("Overriding environment {}".format(id))
c:\Users\37103\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding enviro

hyper-parameters

In [2]:
env = gym.make("FrozenLake-v1", is_slippery=False)
# env = gym.make('MountainCar-v0')

#* hyper-parameters
# ACTION 0: left; 1: down; 2: right; 3: up
num_actions = env.action_space.n
num_states = env.observation_space.n
max_episodes = 10000 # 10000
max_steps_per_episode = 99 # 99

alpha = 0.85
lambd = 0.99
epsilon = 0.1

In [3]:
# def to_one_hot(i, n_classes):
#     one_hot = np.zeros(n_classes)
#     one_hot[i] = 1
#     return one_hot

Q network

In [4]:
def create_q_model(inputs_shape):
    # Network defined by the Deepmind paper
    inputs = keras.Input(shape=(inputs_shape), name='observation')
    Q_value = layers.Dense(num_actions, name='Q_value')(inputs)

    return keras.Model(inputs=inputs, outputs=Q_value)

# Define Q_network, state=16
Q_network = create_q_model(num_states)

Train

In [5]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

for i in range(max_episodes):
    state = env.reset()
    state = tf.one_hot([state], depth=num_states)
    total_reward = 0
    # Play game
    for j in range(max_steps_per_episode):
        # Q_table for 4 actions
        # state = np.asarray([to_one_hot(state, num_states)], dtype=np.float32)
        all_Q = Q_network(state).numpy()
        
        if np.random.rand(1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(all_Q, axis=1)
        
        next_state, reward, done, _ = env.step(int(action))
        # env.render()
        
        next_state = tf.one_hot([next_state], depth=num_states)
        next_all_Q = Q_network(next_state).numpy()

        #? Q'(s,a) <- Q(s,a) + alpha(r + lambd * maxQ(s', a') - Q(s, a))
        target_Q = all_Q
        target_Q[0, action] = reward + lambd * np.max(next_all_Q)
        
        with tf.GradientTape() as tape:
            tape.watch(state)
            qvalues = Q_network(state)
            loss = tf.keras.losses.mean_squared_error(target_Q, qvalues)
        
        weights = Q_network.trainable_weights
        grad = tape.gradient(loss, weights)
        # print(grad)
        optimizer.apply_gradients(zip(grad, weights))
        # 使用参数theta对最优动作价值函数近似
        
        state = next_state
        total_reward += reward
        
        if done:
            epsilon = 1. / ((i / 50) + 10)
            break
    while(i % 100 == 0):    
        print('Episode: {}/{}, Loss: {}, total_reward: {}'.format(i+1, max_episodes, loss, total_reward))
    


Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_reward: 0.0
Episode: 1/10000, Loss: [0.06558661], total_rewa

Test

---

In [ ]:
[var.name for var in tape.watched_variables()]


['Q_value/kernel:0', 'Q_value/bias:0']